# Student name : Wala'a Khalili
# Student number: 403236

######  I have a twitter acount and inside the acount I initiate an app where i have assigned the dataset to be inside  "https://api.twitter.com/1.1/search/tweets.json" , all the retrieved data pulled from that url  that make using Tweepy library easy for processing twitter streaming, it's well documented


In [1]:
# prepare all the needed libraries 
import json
from tweepy import Stream
from itertools import islice
import time
import numpy as np 
import tweepy  
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from datetime import datetime



In [2]:
# credentials from https://apps.twitter.com/. make the authuntication using my consumer_key and token keys 
consumer_key = "***************************"
consumer_secret = "************************"
access_key = "*************************"
access_secret = "***************************************"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

api =  tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


In [42]:
""" Twitter is designed in such a way which only aimed to post the tweets based on an event /user/keyword.
the research should be confined to any particular domain.because of that I specified the search term to "great"
even when I added the code tweepy.stream which is a continous stream I should also specify the search word in (track)
 """
#percentage of tweets mentioning “Yes” during 2 minutes, from all tweets, without
#restrictions.


search_words = "great"
date_since = "2020-05-22"
max_tweets = 2000 # for the last 2000 tweets 
start_time= datetime.now()
yes_count=0

# percentage of tweets mentioning “Yes” during 2 minutes, from all tweets, without restrictions.
text=[]
for tweet in tweepy.Cursor(api.search,
                           q=search_words,
                           result_type=" recent",
                           include_entities=True,
                           lang=" ",
                           untill=date_since).items():
#     print (tweet.created_at, tweet.text)
#     print(tweet.text)
    text.append(tweet.text)
    end_time = datetime.now()
    if np.ceil((end_time - start_time).seconds/60)>=2:  # two minutes 
        break


Rate limit reached. Sleeping for: 780


In [46]:
print(len(text))

449


In [47]:
# we need to count the pearcentage of appearning "yes" in the text

yes_count=0
for item in text:
    index= item.split()
    if index.count("yes") or index.count("Yes"):
        yes_count+=1
#number of counted yes 
print(yes_count)

# percentage of tweets mentioned yes  

yes_percentage= 100*(yes_count/len(text))
print(yes_percentage,"%")

1
0.22271714922048996 %


In [50]:
# sliding window on the same text that have been generated form the query above

def slide_window(seq, n=300):
                  
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        if "yes" in elem:
            result = result[1:] + (elem,)
            yield result
        
windowed= slide_window(text,300)

In [51]:
print(100*(len(list(windowed))/len(text)),"%")


0.22271714922048996 %


In [7]:
# (batching) percentage of tweets mentioning “No” during 2 minutes. Assume that the
# counting function has a very slow rate, use buffers (length 100) for the elements and
# compute the query answer using each tweet once.


""" tweepy has a magnificent attributes that is maximum number of statuses which act as a buffer.
for continuous query we let the code sleep for (time.sleep(920)) for 15 minutes and ask again.
for data we can get data from where we have stopped by using since_id and max_id these attributes helps
getting data from where we have stopped

"""

search_words = "great"
date_since = "2020-05-25"
max_tweets =100 # buffering for 100 
min_id= 1125490788736032770            # assumed number 
maximum_id=2125490788736032770       # assumed number 

start_time = datetime.now()

# percentage of tweets mentioning “Yes” during 2 minutes, from all tweets, without restrictions.
# text=[]
no_count=0
no_percentage=0
for tweet in tweepy.Cursor(api.search,
                           q=search_words,
                           count=5,
                           result_type="recent",
                           include_entities=True,
                           lang="en",
                           since_id=min_id ,
                           max_id=maximum_id,
                           untill=date_since).items(max_tweets):
    text= tweet.text
    
    index = text.split()
    if index.count("no") or index.count("No"):
         no_count+=1

    no_percentage+= 100*(no_count/len(text))
    
    end_time = datetime.now()
    if np.ceil((end_time - start_time).seconds/60)==2:  # retrieve data for two minutes 
        break 
#         time.sleep(15*60)
#         start_time = datetime.now()
#         end_time = datetime.now()
          
    maximum_id=min_id
    min_id+=20000      # assumed number 
    
print(no_percentage,"%")    

40.209423154733535 %


In [39]:
""" (sampling) percentage of tweets mentioning “Hi” during 2 minutes. Assume that the update function is slow.
Update the list with a sample of the elements. (e.g., one in one hundred).
 """

# Define the search term and the date_since date as variables
search_words = "great"
# date_since = "2020-05-20"
max_tweets = 2000 # for the last 2000 tweets 

# percentage of tweets mentioning “Yes” during 2 minutes, from all tweets, without restrictions.
text=[]
n=10
HI_count=0
count=0
start_time=datetime.now()
for tweet in tweepy.Cursor(api.search,
                           q=search_words,
                           result_type=" recent",
                           include_entities=True,
                           lang=" ",
                           until=" ").items():
#     print (tweet.created_at, tweet.text)
#     print(tweet.text)

# take samples one for every one hundred tweet
    if count%n == 0:
        text.append(tweet.text)
    count+=1
    end_time = datetime.now()
    if np.ceil((end_time - start_time).seconds/60)>=2:  # two minutes 
        break



In [40]:
print(len(text))

223


In [41]:
# counting Hi mentioned in the text 
Hi_count=0
for item in text:
    index = item.split()
    if index.count("hi") or index.count("Hi"):
        Hi_count+=1

Hi_percentage= 100*(Hi_count/len(text))
print(Hi_percentage,"%")

0.8968609865470852 %


# Reference 
A treasure map to Twitter Data via Tweepy https://medium.com/analytics-vidhya/a-treasure-map-to-twitter-data-via-tweepy-b7e3d624d88d